In [1]:
import os
import sys
import subprocess
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import numpy as np
import xml.etree.ElementTree as ET
import logging
import warnings
from abc import ABC, abstractmethod
import pandas as pd

In [2]:
class MapOx:
    """
    A class for working with OpenStreetMap data in Python using the OSMnx library.

    Attributes:
        _G (networkx.MultiDiGraph): The graph representing the street network.
        _osm_file_name (str): The name of the OSM file used to create the graph.
        _graphml_file_name (str): The name of the GraphML file used to save the graph.

    Methods:
        save_map_osm(file_name: str) -> None:
            Saves the graph as an OSM file.
        save_map_graphml(file_name: str) -> None:
            Saves the graph as a GraphML file.
    """
    def __init__(self):
        """
        Initializes a new MapOx object.
        """
        try:
            self._G = None
            self._osm_file_name = ""
            self._graphml_file_name = ""
        except Exception as error:
            print("Error in MapOx constructor")
            logging.error(error)

    def save_map_osm(self, file_name):
        """
        Saves the graph as an OSM file.

        Args:
            file_name (str): The name of the output file.
        """
        self._file_osm_name = file_name
        try:
            utn = ox.settings.useful_tags_node
            oxna = ox.settings.osm_xml_node_attrs
            oxnt = ox.settings.osm_xml_node_tags
            utw = ox.settings.useful_tags_way
            oxwa = ox.settings.osm_xml_way_attrs
            oxwt = ox.settings.osm_xml_way_tags
            utn = list(set(utn + oxna + oxnt))
            utw = list(set(utw + oxwa + oxwt))
            ox.settings.all_oneway = True
            ox.settings.useful_tags_node = utn
            ox.settings.useful_tags_way = utw
            ox.settings.all_oneway = True
            ox.save_graph_xml(self._G, filepath=file_name)
        except Exception as error:
            print("Graph convertion to OSM failed. Function save_map_osm")
            raise error

    def save_map_graphml(self, file_name):
        """
        Saves the graph as a GraphML file.

        Args:
            file_name (str): The name of the output file.
        """
        self._graphml_file_name = file_name
        try:
            ox.save_graphml(self._G, file_name)
        except Exception as error:
            print("Graph convertion to GRAPHML failed. Function save_map_graphml")
            raise error

    @property
    def graph(self):
        """
        Returns the graph representing the street network.

        Returns:
            networkx.MultiDiGraph: The graph representing the street network.
        """
        return self._G

    @graph.setter
    def graph(self, value):
        """
        Sets the graph representing the street network.

        Args:
            value (networkx.MultiDiGraph): The new graph representing the street network.
        """
        self._G = value

In [3]:
class MapReader:
    """
    A class for reading OpenStreetMap data and creating a MapOx object.

    Attributes:
        _ox_map (MapOx): The MapOx object representing the street network.

    Methods:
        get_map() -> MapOx:
            Returns the MapOx object representing the street network.
    """
     
    def __init__(self, value, arg="coordinates"):
        """
        Initializes a new MapReader object.

        Args:
            value: The value used to create the MapOx object.
            arg (str): The type of value used to create the MapOx object. Can be "coordinates" or "ox_graphml".
        """
        try:
            self._ox_map = None
            get_ox_map = {
                "place_name": self._map_place_name,
                "coordinates": self._map_coordinates,
                "ox_graphml": self._map_ox_graphml,
            }

            if arg in get_ox_map:
                get_ox_map[arg](value)
            else:
                raise Exception("No valid argument, can be 'coordinates or ox_graph.'")
        except Exception as error:
            print("Error in MapReader constructor")
            logging.error(error)

    def _map_place_name(self, value):
        """
        Creates a MapOx object from a place name.

        Args:
            value: The place name used to create the MapOx object.
        """
        try:
            self._ox_map = MapOx()
            self._get_map_from_place_name(value)
        except Exception as error:
            raise error
    
    def _get_map_from_place_name(self, place_name):
        """
        Creates a MapOx object from a place name.

        Args:
            place_name: The place name used to create the MapOx object.
        """
        try:
            self._ox_map.graph = ox.graph_from_place(place_name, network_type='drive')
            return True
        except Exception as error:
            print("Map G creation error. Function _get_map_from_place_name")
            raise error

    def _map_coordinates(self, value):
        """
        Creates a MapOx object from a set of coordinates.

        Args:
            value: The coordinates used to create the MapOx object.
        """
        try:
            self._ox_map = MapOx()
            self._get_map_from_coordinates(value)
        except Exception as error:
            raise error

    def _map_ox_graphml(self, value):
        """
        Creates a MapOx object from a GraphML file.

        Args:
            value: The path to the GraphML file used to create the MapOx object.
        """
        try:
            self._ox_map = MapOx()
            self._get_map_from_graphml(value)
        except Exception as error:
            raise error

    def _get_map_from_coordinates(self, coordinates):
        """
        Creates a MapOx object from a set of coordinates.

        Args:
            coordinates: The coordinates used to create the MapOx object.
        """
        try:
            west, south, east, north = coordinates
            self._ox_map.graph = ox.graph_from_bbox(
                north, south, east, west, network_type="drive"
            )
            return True
        except Exception as error:
            print("Map G creation error. Function _get_map_from_coordinates")
            raise error

    def _get_map_from_graphml(self, path):
        """
        Creates a MapOx object from a GraphML file.

        Args:
            path: The path to the GraphML file used to create the MapOx object.
        """
        try:
            self._ox_map.graph = ox.io.load_graphml(path)
            return True
        except Exception as error:
            print("Map G creation error. Function _get_map_from_graphml")
            raise error

    def get_map(self):
        """
        Returns the MapOx object representing the street network.

        Returns:
            MapOx: The MapOx object representing the street network.
        """
        return self._ox_map

In [4]:
class Route(ABC):
    """
    An abstract base class for creating routes.

    Attributes:
        _route (list): The list of edges representing the route.

    Methods:
        create_route(G):
            Creates a route on the given graph.
    """
    def __init__(self):
        self._route = []

    @abstractmethod
    def create_route(self):
        """
        Creates a route on the given graph.

        Args:
            G: The graph on which to create the route.
        """
        pass

    @property
    def route(self):
        """
        Returns the list of edges representing the route.

        Returns:
            list: The list of edges representing the route.
        """
        return self._route


class RouteOx(Route):
    """
    A class for creating routes using OpenStreetMap data.

    Attributes:
        dest_name (str): The name of the destination.
        init_orig (tuple): The initial origin coordinates.
        init_dest (tuple): The initial destination coordinates.
        orig (int): The nearest node to the initial origin coordinates.
        dest (int): The nearest node to the initial destination coordinates.
        orig_dist (float): The distance from the initial origin coordinates to the nearest node.
        dest_dist (float): The distance from the initial destination coordinates to the nearest node.

    Methods:
        create_route(G):
            Creates a route on the given graph.
    """

    def __init__(self, dest_name, init_orig, init_dest):
        """
        Initializes a new RouteOx object.

        Args:
            dest_name (str): The name of the destination.
            init_orig (tuple): The initial origin coordinates.
            init_dest (tuple): The initial destination coordinates.
        """
        super().__init__()
        self._dest_name = dest_name
        self._init_orig = init_orig
        self._init_dest = init_dest
        self._orig = None
        self._dest = None
        self._orig_dist = None
        self._dest_dist = None

    def create_route(self, G):
        """
        Creates a route on the given graph.

        Args:
            G: The graph on which to create the route.
        """
        long = self._init_orig[0]
        lat = self._init_orig[1]
        long_dest = self._init_dest[0]
        lat_dest = self._init_dest[1]
        self._orig, self._orig_dist = ox.distance.nearest_nodes(
            G, X=long, Y=lat, return_dist=True
        )
        self._dest, self._dest_dist = ox.distance.nearest_nodes(
            G, X=long_dest, Y=lat_dest, return_dist=True
        )
        self._route = ox.shortest_path(G, self._orig, self._dest, weight="length")


class RouteSumo(Route):
    """
    A class for creating routes using SUMO data.

    Methods:
        create_route(df, ox_route, netReader):
            Creates a route on the given SUMO network.

        _get_nodes_to_from(df, ox_route, positive=True):
            Returns the nodes corresponding to the given edges.

        _convert_nodes_to_sumo_edge(node_from, node_to, netReader):
            Converts the given nodes to a SUMO edge.
    """
    def __init__(self):
        """
        Initializes a new RouteSumo object.
        """
        super().__init__()
        self.ox_route = None

    def create_route(self, df, ox_route, netReader):
        """
        Creates a route on the given SUMO network.

        Args:
            df: The SUMO network data.
            ox_route: The Osmnx route.
            netReader: The SUMO network reader.
        """
        node_from = node_to = None
        node_dfrom = node_dto = None
        node_from, node_to = self._get_nodes_to_from(df, ox_route)
        if node_from:
            node_dfrom, node_dto = self._get_nodes_to_from(df, ox_route, positive=False)

        edge_from = edge_to = None
        if node_dfrom:
            edge_from = self._convert_nodes_to_sumo_edge(node_from, node_to, netReader)
            edge_to = self._convert_nodes_to_sumo_edge(node_dfrom, node_dto, netReader)

        if edge_from and edge_to:
            optPath = netReader.getOptimalPath(edge_from, edge_to)
            for edge in optPath[0]:
                self._route.append(edge.getID())

    def _get_nodes_to_from(self, df, ox_route, positive=True):
        """
        Returns the nodes corresponding to the given edges.

        Args:
            df: The SUMO network data.
            ox_route: The Osmnx route.
            positive (bool): Whether to search for nodes in the positive or negative direction.

        Returns:
            tuple: The nodes corresponding to the given edges.
        """
        idx = 0 if positive else -1
        idx_next = idx + 1 if positive else idx - 1
        node_from = node_to = None
        while True:
            if positive:
                filtered_df = df[df["key"].str.contains(str(ox_route[idx]))]
                filtered_df1 = df[df["key"].str.contains(str(ox_route[idx_next]))]
            else:
                filtered_df = df[df["key"].str.contains(str(ox_route[idx_next]))]
                filtered_df1 = df[df["key"].str.contains(str(ox_route[idx]))]

            if not filtered_df.empty and not filtered_df1.empty:
                if not filtered_df["key"].equals(filtered_df1["key"]):
                    df_key = filtered_df.loc[:, "value"]
                    node_from = df_key.tolist()[0]
                    df1_key = filtered_df1.loc[:, "value"]
                    node_to = df1_key.tolist()[0]
                    return node_from, node_to
                else:
                    idx = idx + 1 if positive else idx - 1
                    idx_next = idx + 1 if positive else idx - 1
            else:
                return None, None

    def _convert_nodes_to_sumo_edge(self, node_from, node_to, netReader):
        """
        Converts the given nodes to a SUMO edge.

        Args:
            node_from: The starting node.
            node_to: The ending node.
            netReader: The SUMO network reader.

        Returns:
            Edge: The SUMO edge corresponding to the given nodes.
        """
        edges_from = node_from.getOutgoing()
        edges_to = node_to.getIncoming()
        edge = None
        edges_ids = []
        for edge_from in edges_from:
            edges_ids.append(edge_from.getID())
            if edge_from in edges_to:
                edge = edge_from

        if edge is None:
            for edge_to in edges_to:
                edges_ids.append(edge_to.getID())

            while True:
                for edge0 in edges_ids:
                    for edge1 in edges_ids:
                        if edge0 != edge1:
                            if edge0 in edge1:
                                edge = netReader.getEdge(edge0)
                                return edge
                            elif edge1 in edge0:
                                edge = netReader.getEdge(edge1)
                                return edge
                return edge
        return edge
    
    @property
    def ox_route(self):
        """
        Returns the Osmnx route.

        Returns:
            list: The Osmnx route.
        """
        return self._route_ox

    @ox_route.setter
    def ox_route(self, ox_route):
        """
        Sets the Osmnx route.

        Args:
            value (list): The new Osmnx route.
        """
        self._route_ox = ox_route

In [5]:
class RoutesFactory(ABC):
    """
    An abstract base class for creating routes.

    Attributes:
        _routes (list): The list of created routes.

    Methods:
        create_routes(Map):
            Creates routes on the given map.
    """
    def __init__(self):
        self._routes = []

    @abstractmethod
    def create_routes(self):
        """
        Creates routes on the given map.

        Args:
            Map: The map on which to create the routes.
        """
        pass

    @property
    def routes(self):
        """
        Returns the list of created routes.

        Returns:
            list: The list of created routes.
        """
        return self._routes

    def get_routes_length(self):
        """
        Returns the number of routes in the routes list.

        Returns:
            int: The number of routes in the routes list.
        """
        return len(self._routes)


class RoutesFactoryOx(RoutesFactory):
    """
    A class for creating routes using OpenStreetMap data.

    Attributes:
        n_rutas (int): The number of routes to create.
        destinations (list): The list of possible destinations.
        seed (int): The random seed to use.

    Methods:
        _sample_points(G):
            Samples points on the given graph.
        _get_destinations_coordinates(_G):
            Returns the coordinates of the destinations.
        create_routes(Map):
            Creates routes on the given map.
    """
    def __init__(self, n_rutas, destinations, seed):
        """
        Initializes a new RoutesFactoryOx object.

        Args:
            n_rutas (int): The number of routes to create.
            destinations (list): The list of possible destinations.
            seed (int): The random seed to use.
        """
        super().__init__()
        self.seed = seed
        np.random.seed(seed)
        self.n_rutas = n_rutas
        self.destinations = destinations

    def _sample_points(self, G):
        """
        Samples points on the given graph.

        Args:
            G: The graph on which to sample points.

        Returns:
            Series: The sampled points.
        """
        n = self.n_rutas
        if nx.is_directed(G):
            warnings.warn(
                "graph should be undirected to not oversample bidirectional edges"
            )

        gdf_edges = ox.utils_graph.graph_to_gdfs(G, nodes=False)[["geometry", "length"]]
        weights = gdf_edges["length"] / gdf_edges["length"].sum()
        idx = np.random.choice(gdf_edges.index, size=n, p=weights)
        lines = gdf_edges.loc[idx, "geometry"]
        return lines.interpolate(np.random.rand(n), normalized=True)

    def _get_destinations_coordinates(self, _G):
        """
        Returns the coordinates of the destinations.

        Args:
            _G: The graph on which to get the destinations.

        Returns:
            list: The coordinates of the destinations.
        """
        G = _G
        # Re-proyectar el grafo a un CRS proyectado
        G2 = ox.projection.project_graph(G)
        G3 = ox.utils_graph.get_undirected(G2)

        points = self._sample_points(G3)

        # Convertir a coordenadas
        points_to_sg = ox.projection.project_gdf(points, to_latlong=True)
        points_frame = points_to_sg.to_frame()
        points_frame = points_frame.reset_index()
        points_coordinates = [(point.x, point.y) for point in points_frame[0]]
        return points_coordinates

    def create_routes(self, Map: MapOx):
        """
        Creates routes on the given map.

        Args:
            Map: The map on which to create the routes.
        """
        _G = Map.graph
        points_coordinates = self._get_destinations_coordinates(_G)
        dest = self.destinations
        # Obtener rutas origen-destino
        for point in points_coordinates:
            dest_aleatorio = np.random.choice(len(dest))
            dest_name = dest[dest_aleatorio][0]
            dest_coord = dest[dest_aleatorio][1]
            route = RouteOx(dest_name, point, dest_coord)
            route.create_route(_G)
            ids_route = route.route
            if not (ids_route) is None:
                if len(ids_route) > 6:
                    self._routes.append(route)
    

class RoutesFactorySumo(RoutesFactory):
    """
    A class for creating routes using SUMO data.

    Methods:
        create_routes(file_net, ox_routes):
            Creates routes on the given SUMO network.
    """
    def __init__(self):
        """
        Initializes a new RoutesFactorySumo object.
        """
        super().__init__()

    def create_routes(self, file_net, ox_routes):
        """
        Creates routes on the given SUMO network.

        Args:
            file_net: The SUMO network file.
            ox_routes: The osmnx routes.
        """
        import sumolib.net as snet

        netReader = snet.readNet(file_net)
        nodes_dict = netReader.getNodesDict()
        df = pd.DataFrame(list(nodes_dict.items()), columns=["key", "value"])

        for ox_route in ox_routes:
            route = ox_route.route
            route_sumo = RouteSumo()
            route_sumo.create_route(df, route, netReader)
            route_sumo_f = route_sumo._route
            if route_sumo_f:
                route_sumo.ox_route = ox_route
                self._routes.append(route_sumo)                

In [6]:
class SumoFiles:
    """
    A class for working with SUMO files.

    Methods:
        save_net_file(osm_file, output_directory=None):
            Saves a SUMO network file from an OpenStreetMap file.
        save_rou_file(route_data, file_name):
            Saves a SUMO routes file from a list of routes.
    """
    def __init__(self):
        """
        Initializes a new SumoFiles object.
        """
        try:
            self._add_path_sumo()
        except Exception as error:
            print("Error in constructor")
            logging.error(error)

    def _add_path_sumo(self):
        """
        Adds the SUMO tools path to the system path.

        Raises:
            Exception: If the SUMO_HOME environment variable is not set or the SUMO tools path does not exist.
        """
        path_sumo_tools = os.path.join(os.environ["SUMO_HOME"], "tools")
        if not os.path.exists(path_sumo_tools):
            try:
                sys.path.append(path_sumo_tools)
            except Exception as error:
                print(
                    "SUMO_HOME is not an environment variable. Function _add_path_sumo"
                )
                raise error

    @staticmethod
    def save_net_file(osm_file, output_directory=None):
        """
        Saves a SUMO network file from an OpenStreetMap file.

        Args:
            osm_file (str): The path to the OpenStreetMap file.
            output_directory (str): The path to the output directory. If None, the network file is saved in the same directory as the OpenStreetMap file.

        Returns:
            str: The path to the saved network file.

        Raises:
            Exception: If the osmBuild module is not installed or the network file creation fails.
        """
        try:
            import osmBuild

            prefix = osm_file.replace(".osm.xml", "")
            args = [
                "--osm-file",
                f"{osm_file}",
                "--vehicle-classes=road",
                f"--prefix={prefix}",
            ]
            if output_directory:
                args = [
                    "--osm-file",
                    f"{osm_file}",
                    "--vehicle-classes=road",
                    f"--prefix={prefix}",
                    f"--output-directory={output_directory}",
                ]
            osmBuild.build(args)
            _file_net_name = prefix + ".net.xml"
            return _file_net_name
        except Exception as error:
            print("Sumo dictionary creation failed. Function save_net_file")
            raise error

    @staticmethod
    def save_rou_file(vehicle_types, vehicles, file_name):
        """
        Saves the specified vehicle types and vehicles to a SUMO routes file.

        Args:
            vehicle_types (List[VehicleType]): The vehicle types to save to the file.
            vehicles (List[Vehicle]): The vehicles to save to the file.
            file_name (str): The name of the file to save the data to.

        Returns:
            bool: True if the file was saved successfully, False otherwise.
        """
        root = ET.Element(
            "routes",
            {
                "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
                "xsi:noNamespaceSchemaLocation": "http://sumo.dlr.de/xsd/routes_file.xsd",
            },
        )
        
        for veh_type in vehicle_types:
            veh_type_keys = veh_type.get_keys()
            vtype = ET.SubElement(root, "vType")
            for key in veh_type_keys:
                vtype.set(key, str(veh_type.get_attribute(key)))

        for vehicle in vehicles:
            vehicle_keys = vehicle.get_keys()
            veh_xml = ET.SubElement(root, "vehicle")
            for key in vehicle_keys:
                if key == "route":
                    route = ET.SubElement(veh_xml, "route")
                    veh_route = vehicle.get_attribute(key)
                    veh_route_sumo = veh_route.route
                    route.set("edges", " ".join(map(str, veh_route_sumo)))
                else:
                    veh_xml.set(key, str(vehicle.get_attribute(key)))
                
        tree = ET.ElementTree(root)
        tree.write(file_name, xml_declaration=True)
        return True

In [7]:
class DynamicAttributes:
    """
    A base class that provides methods for getting and setting dynamic attributes.
    """
    def __init__(self):
        self._attributes = {}

    def get_attribute(self, key):
        """
        Returns the value of the specified attribute.

        Args:
            key (str): The name of the attribute.

        Returns:
            Any: The value of the attribute, or None if the attribute does not exist.
        """
        return self._attributes.get(key)

    def set_attribute(self, key, value):
        """
        Sets the value of the specified attribute.

        Args:
            key (str): The name of the attribute.
            value (Any): The value to set the attribute to.
        """
        self._attributes[key] = value

    def get_keys(self):
        """
        Returns a list of the names of all attributes.

        Returns:
            List[str]: A list of the names of all attributes, or None if there are no attributes.
        """
        if self._attributes:
            return list(self._attributes.keys())
        else:
            return None

class VehicleType(DynamicAttributes):
    """
    Represents a vehicle type with dynamic attributes.
    """
    pass

class Vehicle(DynamicAttributes):
    """
    Represents an individual vehicle with dynamic attributes.
    """
    pass

class VehicleFactory:
    """
    A factory class for creating vehicles.
    """
    def __init__(self):
        self._vehicles = []

    def _create_vehicle(self, id, veh_type, route, depart=0):
        """
        Creates a new Vehicle object with the specified attributes and adds it to the list of vehicles.

        Args:
            id (str): The ID of the vehicle.
            veh_type (str): The ID of the vehicle type.
            route (Any): The route of the vehicle.
            depart (float, optional): The departure time of the vehicle. Defaults to 0.
        """
        vehicle = Vehicle()
        vehicle.set_attribute("id", id)
        vehicle.set_attribute("type", veh_type)
        vehicle.set_attribute("route", route)
        vehicle.set_attribute("depart", depart)
        vehicle.set_attribute("personNumber", 0)
        self._vehicles.append(vehicle)

    def _exponential_distribution(self, routes, veh_types, scale):
        """
        Creates vehicles with exponential distribution of departure times.

        Args:
            routes (List[Any]): The routes to create vehicles for.
            veh_types (List[VehicleType]): The vehicle types to use for creating vehicles.
            scale (float): The scale parameter for the exponential distribution.
        """
        size = len(routes)
        depart = np.random.exponential(scale, size)
        order_depart = np.sort(depart)
        for idx, route in enumerate(routes):
            veh_type = np.random.choice(veh_types)
            self._create_vehicle(f"veh{idx}", veh_type.get_attribute('id'), route, order_depart[idx])

    def _none_distribution(self, ox_routes, veh_types):
        """
        Creates vehicles with no distribution of departure times.

        Args:
            ox_routes (List[Any]): The routes to create vehicles for.
            veh_types (List[VehicleType]): The vehicle types to use for creating vehicles.
        """
        for idx, route in enumerate(ox_routes):
            veh_type = np.random.choice(veh_types)
            self._create_vehicle(f"veh{idx}", veh_type, route)

    def create_vehicles(self, veh_types, routes, scale=0.1, distribution=None):
        """
        Creates vehicles with the specified vehicle types and routes.

        Args:
            veh_types (List[VehicleType]): The vehicle types to use for creating vehicles.
            routes (List[Any]): The routes to create vehicles for.
            scale (float, optional): The scale parameter for the exponential distribution. Defaults to 0.1.
            distribution (str, optional): The distribution to use for departure times. Can be "exponential" or None. Defaults to None.
        """
        if distribution == "exponential":
            self._exponential_distribution(routes, veh_types, scale)
        else:
            self._none_distribution(routes, veh_types)

    @property
    def vehicles(self):
        """
        Returns a list of all vehicles created by the factory.

        Returns:
            List[Vehicle]: A list of all vehicles created by the factory.
        """
        return self._vehicles

    def get_vehicles_length(self):
        """
        Returns the number of vehicles created by the factory.

        Returns:
            int: The number of vehicles created by the factory.
        """
        return len(self._vehicles)

In [8]:
place_name = "Gustavo A. Madero"
mapa = MapReader(place_name, arg = "place_name")
ox_map = mapa.get_map()
G = ox_map.graph
destinations = [["metro Bosque de Aragon", (-99.06923802026726, 19.458256477112243)],
                ["metro Oceania", (-99.08684259000104, 19.445016444249557)],
                ["metro Talisman", (-99.1080774865267, 19.474308308243668)],
                ]

In [9]:
# # west, south, east, north
# coordinates = (-99.165802, 19.383783, -99.103231, 19.428955)
# mapa = MapReader(coordinates)
# ox_map = mapa.get_map()
# G = ox_map.graph
graphm_file = "testGustavo.graphml.xml"
osm_file = "testGustavo.osm.xml"
ox_map.save_map_osm(osm_file)
ox_map.save_map_graphml(graphm_file)
# graphml_file = 'testin1.graphml.xml'
# mapa2 = MapReader(graphml_file, 'ox_graphml')
sumo_files = SumoFiles()
sumo_file_net = sumo_files.save_net_file(osm_file)

In [10]:
seed = 10
# longitud, latitud
# destinations = [
#     ["metro Iztacalco", (-99.1122268021, 19.389042825141992)],
#     ["metro Coyuya", (-99.11344828989398, 19.398097930013037)],
#     ["metro Etiopia", (-99.15606229345083, 19.39593952533453)],
#     ["metro Xola", (-99.13780912121953, 19.39525177855882)],
#     ["metro Viaducto", (-99.13690468723141, 19.400976580148026)],
#     ["metro La Viga", (-99.12626179026907, 19.406719772851407)],
#     ["metro Lazaro Cardenas", (-99.14463368224291, 19.407269278889917)],
#     ["metro Cuauhtemoc", (-99.15458599433383, 19.42590770541133)],
#     ["metro Merced", (-99.12521347728297, 19.42553816384818)],
#     ["metro Pino Suarez", (-99.13293005335198, 19.424465379511155)],
#     ["metro San Antonio Abad", (-99.13459337870798, 19.41580055422141)],
# ]
n_rutas = 1000
# mapa created in the above cell
mapa = ox_map
factory = RoutesFactoryOx(n_rutas, destinations, seed)
factory.create_routes(mapa)

In [11]:
ox_routes = factory.routes  # list of routes objects
factory.get_routes_length()

680

In [12]:
factory_sumo = RoutesFactorySumo()
factory_sumo.create_routes(sumo_file_net, ox_routes)

In [13]:
factory_sumo.get_routes_length()

656

In [14]:
routes = factory_sumo.routes
routes

In [15]:
# vehicle_type1 = VehicleType()
# vehicle_type1.set_attribute("id", "type1")
# # vehicle_type1.set_attribute("accel", accel)
# # vehicle_type1.set_attribute("decel", decel)
# # vehicle_type1.set_attribute("length", length)
# # vehicle_type1.set_attribute("minGap", minGap)
# # vehicle_type1.set_attribute("maxSpeed", maxSpeed)
# vehicle_type1.set_attribute("vClass", "passenger")
# # vehicle_type1.set_attribute("emissionClass", emissionClass)
# vehicle_type1.set_attribute("guiShape", "passenger")
# vehicle_type1.set_attribute("personCapacity", "4")

vehicle_types=[]

vehicle_type1 = VehicleType()
vehicle_type1.set_attribute("id", "type1")
vehicle_type1.set_attribute("vClass", "passenger")
vehicle_type1.set_attribute("guiShape", "passenger/wagon")
vehicle_type1.set_attribute("personCapacity", 5)
vehicle_types.append(vehicle_type1)

vehicle_type2 = VehicleType()
vehicle_type2.set_attribute("id", "type2")
vehicle_type2.set_attribute("vClass", "passenger")
vehicle_type2.set_attribute("guiShape", "passenger/sedan")
vehicle_type2.set_attribute("personCapacity", 5)
vehicle_types.append(vehicle_type2)

vehicle_type3 = VehicleType()
vehicle_type3.set_attribute("id", "type3")
vehicle_type3.set_attribute("vClass", "passenger")
vehicle_type3.set_attribute("guiShape", "passenger/hatchback")
vehicle_type3.set_attribute("personCapacity", 5)
vehicle_types.append(vehicle_type3)


In [16]:
vehicle_factory = VehicleFactory()
vehicle_factory.create_vehicles(vehicle_types, routes, distribution="exponential")


In [17]:
vehicle_factory.get_vehicles_length()

656

In [18]:
vehicles = vehicle_factory.vehicles

In [19]:
veh_t = vehicles[0].get_attribute("personNumber")
print(veh_t)

0


In [92]:
vehicle_type1.get_keys()
vehicle_type1.get_attribute("personCapacity")

5

In [52]:
vehicles[0].get_keys()

['id', 'type', 'route', 'depart', 'personNumber']

In [77]:
veh_route = vehicles[0].get_attribute("route")
veh_route.route

['4338',
 '4345',
 '7422',
 '7426',
 '382',
 '119',
 '8063',
 '4284',
 '4286',
 '4288',
 '4291',
 '4292',
 '335',
 '316',
 '8',
 '315',
 '309',
 '293',
 '300',
 '4294',
 '4296',
 '7667',
 '75',
 '4480',
 '4481',
 '4558',
 '4483',
 '4470',
 '4607',
 '7664',
 '4486',
 '4641',
 '4487',
 '4606',
 '4613',
 '4638',
 '4473',
 '4517',
 '4785',
 '4790',
 '4811',
 '3844',
 '4823',
 '4722',
 '4836',
 '4516',
 '4972',
 '4945',
 '4948',
 '4693',
 '4951',
 '4960',
 '4978',
 '4990',
 '4934',
 '5002',
 '4928',
 '4922',
 '4923',
 '5601',
 '5605',
 '7588',
 '6002',
 '4865',
 '4868',
 '5607',
 '5994',
 '6000',
 '5997',
 '5606',
 '8029',
 '8028',
 '6010',
 '7616',
 '5577',
 '5579',
 '5581',
 '6022',
 '8286',
 '6818',
 '6036',
 '5751',
 '5762',
 '5766',
 '5768',
 '5834',
 '5017',
 '8358',
 '5013',
 '6803',
 '5179',
 '6814',
 '6816',
 '8449',
 '8238',
 '8236',
 '8461',
 '7651',
 '5012']

In [20]:
sumo_files2 = SumoFiles()
sumo_files2.save_rou_file(vehicle_types, vehicles, "testF.rou.xml")

True

In [94]:
sumo_binary = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo')
#output = "--full-output"
output1 = "--tripinfo-output"
output2 = "--emission-output"
# Ejecutar la simulación
subprocess.call([sumo_binary, "-c", "filesFOsmnx.sumocfg", output1 , "reporteFOsmnx.xml", output2, "emissionsFOsmnx2.xml"])

0